In [36]:
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy
import cv2

In [88]:
#Setting the directories
train_dir = 'forTrain/'
test_dir = 'forTest/'

# setting up paths for each images in two list test and train
train_images = [train_dir + train_img for train_img in os.listdir(train_dir)]
test_images =  [test_dir + test_img for test_img in os.listdir(test_dir)]

In [89]:
train_images[0]

'forTrain/cat.0.jpg'

In [90]:
# Function to Read image into np arrays, using Cv2 with Cubic interpolation
def Read_Image(image_path):
    image = cv2.imread(image_path,cv2.IMREAD_COLOR)
    return cv2.resize(image,(64,64),interpolation=cv2.INTER_CUBIC)

In [91]:
# Function to create sets of images

def Create_Data(images):
    m = len(images)
    X = np.zeros((m,64,64,3),dtype=np.uint8) # 64 * 64 dimension and 3 --> RGB Channels
    y = np.zeros((1,m))
    
    for index, image in enumerate(images):
        X[index,:] = Read_Image(image)
        
        y[0,index] = 1 if 'dog' in image.lower() else 0
    
    return X,y

In [92]:
train_set_x , train_set_y = Create_Data(train_images)
test_set_x, test_set_y = Create_Data(test_images)

In [93]:
train_set_x.shape[0]

412

In [94]:
train_set_y.shape

(1, 412)

In [95]:
test_set_x.shape[0]

150

### Flattening of np Arrays

In [96]:
train_set_x = train_set_x.reshape(train_set_x.shape[0], 3*64*64).T
test_set_x = test_set_x.reshape(test_set_x.shape[0], -1).T

In [97]:
train_set_x.shape[1]

412

In [98]:
train_set_y.shape

(1, 412)

In [99]:
test_set_x.shape

(12288, 150)

### Normalization of flattened arrays by 255 for RGB channels

In [100]:
train_set_x = train_set_x / 255  #Broadcasting concept here
test_set_x  = test_set_x / 255 

### Activation Function and Propagation Steps

In [101]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [136]:
def forwardPropagation(w,b,X,y):
    m = X.shape[1] #number of training inputs
    
    z = np.dot(w.T,X) + b 
    
    A = sigmoid(z)
    
    cost = (- 1 / m) * np.sum(y * np.log(A) + (1 - y) * (np.log(1 - A)))  # compute cost
    
    dw = (1/m) * (np.dot(X,(A-y).T))
    db = (1 / m) * np.sum(A - y)
    
    grads = {"dw": dw,
             "db": db}
    
    return grads,cost

In [137]:
def optimizeLR(w,b,X,y,num_iterations, learning_rate, print_cost = False):
    
    costs = []
    
    for i in range(num_iterations):
        
        
        # Cost and gradient calculation (≈ 1-4 lines of code)
        ### START CODE HERE ### 
        grads, cost = forwardPropagation(w, b, X, y)
        ### END CODE HERE ###
        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule (≈ 2 lines of code)
        ### START CODE HERE ###
        w = w - learning_rate * dw  # need to broadcast
        b = b - learning_rate * db
        ### END CODE HERE ###
        
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training examples
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" % (i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [138]:
def predict(w,b,X):
    
    m = X.shape[1]
    y_hat = np.zeros((1,m))
    
    w = w.reshape(X.shape[0],1)
    
    a = sigmoid(np.dot(w.T,X) + b)
    
    for y_hati in range(a.shape[1]):
        y_hat[0,y_hati] = 1 if a[0,y_hati] > 0.5 else 0
    
    return y_hat

In [139]:
def model(X_train, Y_train, X_test, Y_test, num_iterations=2000, learning_rate=0.5, print_cost=False):
    w  = np.zeros(shape=(X_train.shape[0], 1))
    
    b = 0
    
    parameters, grads, costs = optimizeLR(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    
    w = parameters['w']
    b = parameters['b']
    
    y_hat_test = predict(w,b,X_test)
    y_hat_train = predict(w,b,X_train)
    
    print("train accuracy: {} %".format(100 - np.mean(np.abs(y_hat_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(y_hat_test - Y_test)) * 100))
    
    
    d = {"costs": costs,
         "y_hat_test": y_hat_test, 
         "y_hat_train" : y_hat_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [140]:
d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 2000, learning_rate = 0.005, print_cost = True)

Cost after iteration 0: 0.693147
Cost after iteration 100: 0.816658
Cost after iteration 200: 0.757037
Cost after iteration 300: 0.702649
Cost after iteration 400: 0.649355
Cost after iteration 500: 0.596808
Cost after iteration 600: 0.545950
Cost after iteration 700: 0.497500
Cost after iteration 800: 0.451523
Cost after iteration 900: 0.407711
Cost after iteration 1000: 0.365783
Cost after iteration 1100: 0.326064
Cost after iteration 1200: 0.292292
Cost after iteration 1300: 0.274997
Cost after iteration 1400: 0.265048
Cost after iteration 1500: 0.255887
Cost after iteration 1600: 0.247337
Cost after iteration 1700: 0.239329
Cost after iteration 1800: 0.231806
Cost after iteration 1900: 0.224724
train accuracy: 97.81553398058253 %
test accuracy: 65.33333333333333 %
